In [2]:
import os
import cv2
import csv

import deepface.DeepFace
import torch
import shutil
import zipfile
import numpy as np
import pandas as pd

from ultralytics import YOLO
from deep import DeepFace

24-09-20 12:14:10 - Directory /home/diano/.deepface has been created
24-09-20 12:14:10 - Directory /home/diano/.deepface/weights has been created


### Less Weight
*Aqui eu tenho a função **CompactDataset** para compactar o dataset e não ficar muito pesado no sistema e a função **DecompressDataset** para descompactar e poder fazer o uso do dataset*

In [3]:
def CompactDataset():
    shutil.make_archive('dataset_compressed', 'zip', './dataset')
    shutil.rmtree('./dataset')

In [4]:
def DecompressDataset():
    if os.path.exists('dataset_compressed.zip'):
        shutil.unpack_archive('dataset_compressed.zip', './dataset')
        os.remove('dataset_compressed.zip')

### Recording and Save Face Moves

*Aqui eu estou utilizando a biblioteca **OpenCV** para fazer a gravação do video onde utilizando a biblioteca **Ultralytics** para importar o **YOLO** e ter acesso ao modelo **yolov8n-face.pt** onde ele faz a dectecção da face da pessoa, se nenhuma face for detectada ele "corta" esse trecho do video, enquanto a face está sendo detectada e o video está sendo gravado e utilizando a biblioteca **OS** eu estou salvando cada frame do video em uma pasta com o nome da pessoa para utilizar isso de Dataset, tendo esse Dataset eu posso comparar 1 imagem da pessoa com o dataset e ver qual o resultado, escolhi fazer comparando 1 imagem com varias para melhorar a precisão do algoritmo*

In [7]:
def RecordVideoAndSaveFrames(username):
    DecompressDataset()
    model = YOLO('yolov8n-face.pt')
    path_video = os.path.join(f'./dataset/{username}/')
    output_video = os.path.join(f'./dataset/{username}/registration.mp4')

    output_frames = os.path.join(f'./dataset/{username}')

    if not os.path.exists(output_frames):
        os.makedirs(output_frames)

    files = os.listdir(output_frames)

    frame_counter = 0

    if len(files) > 0:
        shutil.rmtree(output_frames)
        os.makedirs(output_frames)

    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 60

    if not os.path.exists(path_video):
        os.makedirs(path_video)

    writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))

    while True:
        ret, frame = cap.read()

        results = model.predict(frame, conf=0.7)

        for result in results:
            params = result.boxes

            for box in params:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2),
                h, w = y2 - y1, x2 - x1
                frame_border = frame.copy()

                cv2.rectangle(frame_border, (x1, y1), (x2, y2), (0, 255, 0), 2)
                writer.write(frame)

                frame_counter += 1

                frame_filename = os.path.join(output_frames, f"frame_{frame_counter:04d}.jpg")

                cv2.imwrite(frame_filename, frame)

        cv2.imshow('Video', frame_border)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    writer.release()
    print(f"{frame_counter} frames salvos como imagens .jpg.")
    CompactDataset()
    cv2.destroyAllWindows()

In [9]:
RecordVideoAndSaveFrames('alexandre_diano')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'



0: 480x640 1 face, 30.3ms
Speed: 1.1ms preprocess, 30.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.5ms
Speed: 1.4ms preprocess, 4.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.7ms
Speed: 1.3ms preprocess, 4.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.5ms
Speed: 1.2ms preprocess, 4.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.7ms
Speed: 1.5ms preprocess, 4.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 5.0ms
Speed: 1.8ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.8ms
Speed: 1.5ms preprocess, 4.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.6ms
Speed: 1.5ms preprocess, 4.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 5.

### Face Recognition

*Aqui eu estou utilizando a biblioteca **DeepFace** para fazer o Reconhecimento Facial, a principio eu irei utilizar a função **Find** do modelo, para comparar 1 imagem (que será provida apartir da webcam) com as imagens que temos no Dataset da pessoa, assim tendo uma precisão maior e falsos positivos menores*

In [10]:
def FaceRecognition(img, person):
    try:
        result = DeepFace.find(img_path=img, db_path=f"./dataset/{person}", model_name="Facenet")

        identity = np.array(result[0]['identity'])
        distance = np.array(result[0]['distance'])

        if identity.size > 0:
            return True, np.mean(distance)
        else:
            return False, np.mean(distance)
    except:
        print('exception')

### Verify Access

*Essa função irá abrir a Webcam e se detectar um rosto ele chamará a função **FaceRecognition** passando 1 frame e o identificador de usuario onde essa função caso seja identificado um rosto conhecido ela irá retornar o valor True e a distancia média se não, irá retornar apenas False*

In [1]:
def VerifyAccess(identifier):
    model = YOLO('yolov8n-face.pt')
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        results = model.predict(frame, conf=0.7)

        for result in results:
            params = result.boxes

            for box in params:
                if box:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2),
                    h, w = y2 - y1, x2 - x1
                    frame_border = frame.copy()

                    cv2.rectangle(frame_border, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    verify, distance = FaceRecognition(frame, identifier)
                    cv2.putText(frame_border, f'Acesso: {verify} e Distancia: {distance}', (20, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('Verify Access', frame_border)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [13]:
DecompressDataset()

In [15]:
VerifyAccess('alexandre_diano') 

WARNING ⚠️ 'source' is missing. Using 'source=/home/diano/PastaCheiaDeMerda/University/ProcessamentoDigital/venv/lib/python3.12/site-packages/ultralytics/assets'.

image 1/2 /home/diano/PastaCheiaDeMerda/University/ProcessamentoDigital/venv/lib/python3.12/site-packages/ultralytics/assets/bus.jpg: 640x480 2 faces, 4.9ms
image 2/2 /home/diano/PastaCheiaDeMerda/University/ProcessamentoDigital/venv/lib/python3.12/site-packages/ultralytics/assets/zidane.jpg: 384x640 1 face, 5.2ms
Speed: 1.5ms preprocess, 5.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)


[ WARN:0@1554.147] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1554.148] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


AttributeError: 'NoneType' object has no attribute 'copy'

## Open Camera and invert video

In [ ]:
from ultralytics import YOLO
import cv2
import threading
from deep import DeepFace
import numpy as np

processing = False
access = False
status_message = ''
lock = threading.Lock()

def verify_access(img, person):
    global processing
    global access
    global status_message
    try:
        result = DeepFace.verify(img1_path=img, img2_path=f"./dataset/{person}/frame_0028.jpg", model_name="Facenet")

        with lock:
            if result['verified']:
                print('Acesso permitido')
                access = True
                status_message = 'Acesso permitido'
            else:
                print('Acesso negado')
                access = False
                status_message = 'Acesso negado'
    except Exception as e:
        with lock:
            print(f"Erro na verificação de acesso: {e}")
            access = False
            status_message = 'Erro na verificação'
    finally:
        processing = False

def start_monitoring():
    global processing
    model = YOLO('yolov8n-face.pt')
    cap = cv2.VideoCapture(1)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # frame = cv2.flip(frame, 0)
        frame_border = frame.copy()

        results = model.predict(frame, conf=0.7)

        max_area = 0
        closest_box = None

        for result in results:
            params = result.boxes

            for box in params:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(frame.shape[1], x2)
                y2 = min(frame.shape[0], y2)
                area = (x2 - x1) * (y2 - y1)
                if area > max_area:
                    max_area = area
                    closest_box = (x1, y1, x2, y2)

        if closest_box is not None:
            x1, y1, x2, y2 = closest_box
            with lock:
                if access:
                    color = (255, 0, 255)
                else:
                    color = (255, 0, 0)
            cv2.rectangle(frame_border, (x1, y1), (x2, y2), color, 2)
            face_image = frame[y1:y2, x1:x2]
            if not processing:
                processing = True
                threading.Thread(target=verify_access, args=(face_image, "alexandre_diano",)).start()

        with lock:
            cv2.putText(frame_border, status_message, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Verify Access', frame_border)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    start_monitoring()



0: 480x640 1 face, 4.9ms
Speed: 1.9ms preprocess, 4.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 5.0ms
Speed: 1.7ms preprocess, 5.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.3ms
Speed: 1.1ms preprocess, 4.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 4.3ms
Speed: 1.0ms preprocess, 4.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 23.1ms
Speed: 3.1ms preprocess, 23.1ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 21.0ms
Speed: 2.5ms preprocess, 21.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 18.7ms
Speed: 2.5ms preprocess, 18.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 17.6ms
Speed: 2.4ms preprocess, 17.6ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640

### Valid face before send to firebase

In [12]:
import cv2
from ultralytics import YOLO

def picture_validation(img):
    model = YOLO('yolov8n-face.pt')
    model.classes = [0]

    image = cv2.imread(img)
    if image is None:
        raise FileNotFoundError(f"Não foi possível encontrar ou abrir a imagem: {img}")

    results = model.predict(image, conf=0.7)

    total_faces = 0
    for result in results:
        total_faces += len(result.boxes)

    if total_faces > 0:
        print(f"{total_faces} face(s) detectada(s) na imagem.")
        return True
    else:
        print("Nenhuma face detectada na imagem.")
        return False
    
if __name__ == '__main__':
    picture_validation(img="dataset/alexandre_diano/frame_0028.jpg")


0: 480x640 1 face, 5.2ms
Speed: 2.2ms preprocess, 5.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
1 face(s) detectada(s) na imagem.


In [9]:
import deepface.DeepFace

deepface.DeepFace.stream(db_path="dataset", model_name="Facenet", frame_threshold=2, anti_spoofing=True, enable_face_analysis=False)

24-09-29 20:20:55 - Facenet is built
24-09-29 20:20:56 - freezed
24-09-29 20:21:01 - freeze released
24-09-29 20:21:01 - freezed
24-09-29 20:21:06 - freeze released
24-09-29 20:21:07 - Hello, dataset/alexandre_diano/Photo from 2024-09-28 17-52-10.057728.jpeg
24-09-29 20:21:07 - freezed
24-09-29 20:21:12 - freeze released
24-09-29 20:21:12 - freezed
24-09-29 20:21:17 - freeze released
24-09-29 20:21:18 - Hello, dataset/alexandre_diano/Photo from 2024-09-28 17-52-10.057728.jpeg
24-09-29 20:21:18 - freezed
24-09-29 20:21:23 - freeze released
24-09-29 20:21:23 - Hello, dataset/alexandre_diano/Photo from 2024-09-28 17-52-10.057728.jpeg
24-09-29 20:21:24 - freezed
24-09-29 20:21:29 - freeze released
24-09-29 20:21:29 - Hello, dataset/alexandre_diano/Photo from 2024-09-28 17-52-10.057728.jpeg
24-09-29 20:21:29 - freezed
